In [16]:
import pandas as pd
import os
from glob import glob
import numpy as np
from pandas.api.types import CategoricalDtype

In [17]:
ex1 = pd.read_parquet('test/1.회원정보/2018_회원정보_전체_test.parquet')

In [18]:
all_cols = ex1.columns.tolist()
print("전체 컬럼 이름:")
print(all_cols)
str_cols = ex1.select_dtypes(include=['object']).columns.tolist()
print("\n문자형 컬럼 이름:")
print(str_cols)
num_cols = ex1.select_dtypes(include=[np.number]).columns.tolist()
print("\n숫자형 컬럼 이름:")
print(num_cols)

전체 컬럼 이름:
['기준년월', 'ID', '남녀구분코드', '연령', '회원여부_이용가능', '회원여부_이용가능_CA', '회원여부_이용가능_카드론', '소지여부_신용', '소지카드수_유효_신용', '소지카드수_이용가능_신용', '입회일자_신용', '입회경과개월수_신용', '회원여부_연체', '이용거절여부_카드론', '동의여부_한도증액안내', '수신거부여부_TM', '수신거부여부_DM', '수신거부여부_메일', '수신거부여부_SMS', '가입통신회사코드', '탈회횟수_누적', '최종탈회후경과월', '탈회횟수_발급6개월이내', '탈회횟수_발급1년이내', '거주시도명', '직장시도명', '마케팅동의여부', '유효카드수_신용체크', '유효카드수_신용', '유효카드수_신용_가족', '유효카드수_체크', '유효카드수_체크_가족', '이용가능카드수_신용체크', '이용가능카드수_신용', '이용가능카드수_신용_가족', '이용가능카드수_체크', '이용가능카드수_체크_가족', '이용카드수_신용체크', '이용카드수_신용', '이용카드수_신용_가족', '이용카드수_체크', '이용카드수_체크_가족', '이용금액_R3M_신용체크', '이용금액_R3M_신용', '이용금액_R3M_신용_가족', '이용금액_R3M_체크', '이용금액_R3M_체크_가족', '_1순위카드이용금액', '_1순위카드이용건수', '_1순위신용체크구분', '_2순위카드이용금액', '_2순위카드이용건수', '_2순위신용체크구분', '최종유효년월_신용_이용가능', '최종유효년월_신용_이용', '최종카드발급일자', '보유여부_해외겸용_본인', '이용가능여부_해외겸용_본인', '이용여부_3M_해외겸용_본인', '보유여부_해외겸용_신용_본인', '이용가능여부_해외겸용_신용_본인', '이용여부_3M_해외겸용_신용_본인', '연회비발생카드수_B0M', '연회비할인카드수_B0M', '기본연회비_B0M', '제휴연회비_B0M', '할인금액_기본연회비_B0M', '할인금액_제휴연회비_B0M', '청구금액_기본연회비_B0M', '청구

In [19]:
cols = ['가입통신회사코드', '거주시도명', '직장시도명', '_1순위신용체크구분', '_2순위신용체크구분', '연회비발생카드수_B0M', '상품관련면제카드수_B0M', '임직원면제카드수_B0M', '우수회원면제카드수_B0M', '기타면제카드수_B0M', 'Life_Stage']  # 실제 열 이름으로 바꿔주세요

In [20]:
mapping = {
    'L사': 0,
    'K사': 1,
    'S사': 2
}
ex1['가입통신회사코드'] = (
    ex1['가입통신회사코드']
      .map(mapping)        # L사→0, K사→1, S사→2, 그 외→NaN
      .fillna(-1)          # NaN→-1
      .astype(int)         # 정수형
)

In [21]:
ex1['연령'] = ex1['연령'].astype(str).str.replace('대', '', regex=False)

# 2) 숫자로 변환, 에러는 NaN→fillna(-1)→정수형
ex1['연령'] = (
    pd.to_numeric(ex1['연령'], errors='coerce')
      .fillna(-1)
      .astype(int)
)


In [22]:

regions = [
    '서울', '부산', '대구', '인천', '광주', '대전', '울산', '세종',
    '경기', '강원', '충북', '충남', '전북', '전남', '경북', '경남', '제주'
]

region_type = CategoricalDtype(categories=regions, ordered=True)

for col in ['거주시도명', '직장시도명']:
    ex1[col] = (
        ex1[col]
           .astype(str)                   # 혹시 숫자 타입 섞여 있어도 문자열로
           .astype(region_type)           # 카테고리로 변환
           .cat.codes                     # 0~16, 리스트 외 값은 -1
    )

In [23]:
count_cols = [
    '연회비발생카드수_B0M',
    '상품관련면제카드수_B0M',
    '임직원면제카드수_B0M',
    '우수회원면제카드수_B0M',
    '기타면제카드수_B0M'
]

for col in count_cols:
    ex1[col] = (
        ex1[col]
          .astype(str)  # None → 'None', NaN → 'nan'
          .str.replace('개', '', regex=False)  # '0개' → '0', 'None' → 'None'
    )
    # 숫자로 변환, 에러는 NaN 처리 → NaN → -1 로 채우기 → 정수형
    ex1[col] = (
        pd.to_numeric(ex1[col], errors='coerce')
          .fillna(-1)
          .astype(int)
    )

In [24]:
le_cols = ['_1순위신용체크구분', '_2순위신용체크구분', 'Life_Stage']

for col in le_cols:
    codes, uniques = pd.factorize(ex1[col], sort=True)
    ex1[col] = codes 

In [25]:
all_cols = ex1.columns.tolist()
print("전체 컬럼 이름:")
print(all_cols)
str_cols = ex1.select_dtypes(include=['object']).columns.tolist()
print("\n문자형 컬럼 이름:")
print(str_cols)
num_cols = ex1.select_dtypes(include=[np.number]).columns.tolist()
print("\n숫자형 컬럼 이름:")
print(num_cols)

전체 컬럼 이름:
['기준년월', 'ID', '남녀구분코드', '연령', '회원여부_이용가능', '회원여부_이용가능_CA', '회원여부_이용가능_카드론', '소지여부_신용', '소지카드수_유효_신용', '소지카드수_이용가능_신용', '입회일자_신용', '입회경과개월수_신용', '회원여부_연체', '이용거절여부_카드론', '동의여부_한도증액안내', '수신거부여부_TM', '수신거부여부_DM', '수신거부여부_메일', '수신거부여부_SMS', '가입통신회사코드', '탈회횟수_누적', '최종탈회후경과월', '탈회횟수_발급6개월이내', '탈회횟수_발급1년이내', '거주시도명', '직장시도명', '마케팅동의여부', '유효카드수_신용체크', '유효카드수_신용', '유효카드수_신용_가족', '유효카드수_체크', '유효카드수_체크_가족', '이용가능카드수_신용체크', '이용가능카드수_신용', '이용가능카드수_신용_가족', '이용가능카드수_체크', '이용가능카드수_체크_가족', '이용카드수_신용체크', '이용카드수_신용', '이용카드수_신용_가족', '이용카드수_체크', '이용카드수_체크_가족', '이용금액_R3M_신용체크', '이용금액_R3M_신용', '이용금액_R3M_신용_가족', '이용금액_R3M_체크', '이용금액_R3M_체크_가족', '_1순위카드이용금액', '_1순위카드이용건수', '_1순위신용체크구분', '_2순위카드이용금액', '_2순위카드이용건수', '_2순위신용체크구분', '최종유효년월_신용_이용가능', '최종유효년월_신용_이용', '최종카드발급일자', '보유여부_해외겸용_본인', '이용가능여부_해외겸용_본인', '이용여부_3M_해외겸용_본인', '보유여부_해외겸용_신용_본인', '이용가능여부_해외겸용_신용_본인', '이용여부_3M_해외겸용_신용_본인', '연회비발생카드수_B0M', '연회비할인카드수_B0M', '기본연회비_B0M', '제휴연회비_B0M', '할인금액_기본연회비_B0M', '할인금액_제휴연회비_B0M', '청구금액_기본연회비_B0M', '청구

In [26]:
cols = ['기준년월',
 'ID',
 '남녀구분코드',
 '연령',
 '회원여부_이용가능_CA',
 '회원여부_이용가능_카드론',
 '소지카드수_유효_신용',
 '입회일자_신용',
 '회원여부_연체',
 '이용거절여부_카드론',
 '동의여부_한도증액안내',
 '수신거부여부_TM',
 '수신거부여부_SMS',
 '가입통신회사코드',
 '탈회횟수_누적',
 '탈회횟수_발급6개월이내',
 '거주시도명',
 '마케팅동의여부',
 '유효카드수_신용_가족',
 '유효카드수_체크',
 '이용카드수_체크',
 '이용금액_R3M_신용체크',
 '이용금액_R3M_체크',
 '_1순위신용체크구분',
 '최종유효년월_신용_이용가능',
 '보유여부_해외겸용_본인',
 '연회비발생카드수_B0M',
 '제휴연회비_B0M',
 '카드신청건수',
 'Life_Stage']
ex1 = ex1[cols]

In [27]:
ex1.to_parquet('회원_전처리_test.parquet', index=False)